In [1]:
import sys
sys.path.append("..")

In [2]:
import numpy as np
import pandas as pd
import time

In [3]:
from meta_qa.integrations import BigQueryIntegration
from meta_qa.tools import run_qa_pipeline
from meta_qa.tools import generate_documentation
from meta_qa.tools import generate_tests_report

In [4]:
integration = BigQueryIntegration("idwall-data", "dw_idwall")

In [5]:
metadata = integration.get_metadata()

/home/danilo.lessa/.local/lib/python3.6/site-packages/google/auth/_default.py:66: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [6]:
time1 = time.time()
tests_data = run_qa_pipeline(integration, n_workers=512)
time2 = time.time()

New pandarallel memory created - Size: 2000 MB
Pandarallel will run on 512 workers
Error: {'result': False, 'raw_result': GenericGBQException('Reason: 400 No matching signature for operator < for argument types: INT64, STRING. Supported signatures: ANY < ANY at [4:15]',), 'operator': 'not_window_percentile_null', 'project_id': 'idwall-data', 'dataset': 'dw_idwall', 'table_name': 'ft_bpo_services', 'column_name': 'sla_total'} 

Error: {'result': False, 'raw_result': GenericGBQException('Reason: 400 Name id_status_field_audit not found inside t2 at [4:98]',), 'operator': 'related_to', 'project_id': 'idwall-data', 'dataset': 'dw_idwall', 'table_name': 'ft_bpo_fields', 'column_name': 'id_status_field_audit'} 

Error: {'result': False, 'raw_result': GenericGBQException('Reason: 400 Name id_user not found inside t2 at [4:80]',), 'operator': 'related_to', 'project_id': 'idwall-data', 'dataset': 'dw_idwall', 'table_name': 'ft_bpo_operations', 'column_name': 'id_user'} 

Error: {'result': False


Error: {'result': False, 'raw_result': GenericGBQException('Reason: 400 GET https://www.googleapis.com/bigquery/v2/projects/idwall-data/queries/40f88661-84e6-45d4-8942-9bfe23561fc8?maxResults=0&timeoutMs=900&location=southamerica-east1: division by zero: 0 / 0',), 'operator': 'not_null', 'project_id': 'idwall-data', 'dataset': 'dw_idwall', 'table_name': 'lk_time_day_of_week', 'column_name': 'desc_day_of_week'} 

Error: {'result': False, 'raw_result': GenericGBQException('Reason: 400 GET https://www.googleapis.com/bigquery/v2/projects/idwall-data/queries/a5bd969d-9d49-46a1-8a33-d72178ada408?maxResults=0&timeoutMs=900&location=southamerica-east1: division by zero: 0 / 0',), 'operator': 'not_null', 'project_id': 'idwall-data', 'dataset': 'dw_idwall', 'table_name': 'lk_time_date', 'column_name': 'id_day_of_week'} 

Error: {'result': False, 'raw_result': GenericGBQException('Reason: 400 GET https://www.googleapis.com/bigquery/v2/projects/idwall-data/queries/f55df858-13f3-4ae0-8509-cce4091e

In [7]:
print(time2 - time1)

21.856783151626587


In [8]:
tests_data.shape

(248, 9)

In [9]:
tests_data.head(5)

not_null  \
table_catalog table_schema table_name  column_name                
idwall-data   dw_idwall    ft_attempts created       True (1.0)   
                                       created_date  True (1.0)   
                                       created_hour  True (1.0)   
                                       duration                   
                                       end_date                   

                                                        lower_than  \
table_catalog table_schema table_name  column_name                   
idwall-data   dw_idwall    ft_attempts created            True (0)   
                                       created_date       True (0)   
                                       created_hour  False (77860)   
                                       duration                      
                                       end_date                      

                                                    not_window_percentile_null  \
table_catalog table_schema table_name  column_name                               
idwall-data   dw_idwall    ft_attempts created                                   
                                       created_date                              
                                       created_hour                              
                                       duration                                  
                                       end_date                                  

                                                    larger_than  \
table_catalog table_schema table_name  column_name                
idwall-data   dw_idwall    ft_attempts created                    
                                       created_date               
                                       created_hour               
                                       duration                   
                                       end_date        True (0)   

                                                    not_duplicates  \
table_catalog table_schema table_name  column_name                   
idwall-data   dw_idwall    ft_attempts created                       
                                       created_date                  
                                       created_hour                  
                                       duration                      
                                       end_date                      

                                                    check_stage  \
table_catalog table_schema table_name  column_name                
idwall-data   dw_idwall    ft_attempts created                    
                                       created_date               
                                       created_hour               
                                       duration                   
                                       end_date                   

                                                        related_to  \
table_catalog table_schema table_name  column_name                   
idwall-data   dw_idwall    ft_attempts created                       
                                       created_date                  
                                       created_hour                  
                                       duration      False (error)   
                                       end_date                      

                                                    not_percentile_null  \
table_catalog table_schema table_name  column_name                        
idwall-data   dw_idwall    ft_attempts created                            
                                       created_date                       
                                       created_hour                       
                                       duration              True (0.0)   
                                       end_date                           

                                                    timestamp_sanity  


In [10]:
from jinja2 import Environment, FileSystemLoader, select_autoescape
env = Environment(
    loader=FileSystemLoader(searchpath='templates'),
    autoescape=select_autoescape(['html', 'xml'])
)

## Schema

In [11]:
b = metadata.reset_index()


tbl_cols = ["table_name", "table_title", "table_ignore", "table_description_text"]


projects_data  = b.groupby("table_catalog")

projects = []
for project_name, project_data in projects_data:
    
    ds_data = project_data.drop(columns="table_catalog").groupby("table_schema")
    datasets = []
    
    for ds_name, ds_group in ds_data:
        tbl_data = ds_group.drop(columns="table_schema").groupby("table_name")
        tables = []
        for tbl_name, tbl_group in tbl_data:
            
            col_data = tbl_group.drop(columns=tbl_cols)
            tbl_meta = tbl_group.iloc[0][tbl_cols].to_dict()
            tbl_element = col_data.to_dict(orient='records')
            
            headers = []
            columns = []
            for row in tbl_element:
                headers = row.keys()
                columns.append(row.values())
            
            tables.append({**tbl_meta,
                            'headers': headers,
                            'columns': columns})
            
        datasets.append({'dataset_name': ds_name,
                         'tables': tables})
        
    projects.append({'project_name': project_name,
                     'datasets': datasets})

In [12]:
template = env.get_template('metadata.html')

with open("a.html", "w") as fid:
    fid.write(template.render(projects=projects))

## Tests report

In [13]:
failed_cells = tests_data.applymap(lambda x: "False" in x).astype(bool)

In [14]:
success_cells = tests_data.applymap(lambda x: "True" in x).astype(bool)

In [15]:
failed_rows = failed_cells.sum(axis=1).astype(bool)

In [16]:
success_rows = success_cells.sum(axis=1).astype(bool)

In [17]:
failed_cells_2 = tests_data.copy()
failed_cells_2[~failed_cells] = ""
pd.DataFrame(failed_cells_2.apply(lambda x: x.str.cat(), axis=1)).head(5)

0
table_catalog table_schema table_name  column_name                
idwall-data   dw_idwall    ft_attempts created                    
                                       created_date               
                                       created_hour  False (77860)
                                       duration      False (error)
                                       end_date

In [18]:
def fail_function(row):
    return " | ".join(row.loc[row].index)

In [19]:
failed_table = (pd.DataFrame(failed_cells.apply(fail_function, axis=1))
                                         .rename(columns={0: "failed_asserts"})
                                         .where(lambda x: x["failed_asserts"] != "")
                                         .dropna())

In [20]:
success_table = (pd.DataFrame(success_cells.apply(fail_function, axis=1))
                                         .rename(columns={0: "ok_asserts"})
                                         .where(lambda x: x["ok_asserts"] != "")
                                         .dropna())

In [21]:
new_table = success_table.join(failed_table).fillna("")

In [22]:
failed_dataset = failed_table.groupby(["table_schema", "table_name"]).count()

In [23]:
failed_table.groupby("table_name")

In [24]:
tables_summary = []



In [25]:
projects = []
for proj_name, proj_group in new_table.groupby(["table_catalog"]):
    datasets = []
    for ds_name, ds_group in new_table.groupby(["table_schema"]):
        tables = []
        ds_error_count = ds_group.where(lambda x: x["failed_asserts"] != "").count().values[0]
        for table_name, table_group in new_table.groupby(["table_name"]):
            table_summary = table_group.droplevel([0, 1, 2]).reset_index()
            headers = table_summary.columns.tolist()
            columns = table_summary.values.tolist()
            table_name = table_name
            table_error_count = table_group.where(lambda x: x["failed_asserts"] != "").count().values[0]
            tables.append({"table_name": table_name, 
                           "error_count": str(table_error_count),
                           "headers": headers, 
                           "columns": columns})
        datasets.append({"dataset_name": ds_name,
                         "error_count": str(ds_error_count),
                         "tables": tables})
    projects.append({"project_name": proj_name, "datasets": datasets})

In [26]:
template = env.get_template('metadata.html')
with open("b.html", "w") as fid:
    fid.write(template.render(projects=projects))

In [27]:
projects[0]["datasets"][0]["dataset_name"]

'dw_idwall'

In [28]:
projects[0]["datasets"][0]["error_count"]

'13'